In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential, Model
import random


import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F


from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision import datasets, transforms
sns.set(style='white', context='notebook', palette='deep')

In [61]:
img_folder=r'/content/drive/MyDrive/FDL_kaggle/train_images/train_images'
mask_folder = r'/content/drive/MyDrive/FDL_kaggle/train_masks/train_masks'

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  


In [2]:
image_id = []
for x in os.listdir('train_images'):
    x = x.replace('.jpg','')
    image_id.append(x)

In [4]:
img_folder = 'train_images'
mask_folder = 'train_masks'

In [8]:
# Preparing the data
X = []
y = []
h = 256
w = 256
for i,img_id in enumerate(image_id):
    print(i)
    im_file   =  img_id+".jpg"
    im_path = os.path.join(img_folder,im_file)
    img = Image.open(im_path) # read image in PIL
    img = img.resize((w,h)) # first component is width and then height
    img_array = np.array(img)
    img_array = img_array/255.0
    #img_array = np.moveaxis(img_array, -1, 0)
    #print(img_array.shape)
    
    mask_file =  img_id+".png"
    mask_path = os.path.join(mask_folder,mask_file)
    mask = Image.open(mask_path)
    mask = mask.resize((w,h))
    mask_array = np.array(mask)
    #shape = mask_array.shape
    #mask_array = np.reshape(mask_array,(1,shape[0],shape[1]))
    
    X.append(img_array)
    y.append(mask_array)
    
    
    
X = np.array(X)
y = np.array(y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260


In [9]:
y = np.array(y)
y.max()

26

In [10]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26], dtype=uint8)

In [80]:
X=torch.from_numpy(X)
y=torch.from_numpy(y)

X = X.float()
y = y.to(torch.int64)

print(X.dtype,y.dtype)

torch.float32 torch.int64


In [82]:
from torch.nn.functional import one_hot
y = one_hot(y)

In [84]:
Xt = torch.transpose(X, 1, 3)
yt = torch.transpose(y, 1, 3)

torch.Size([10, 3, 512, 512])

In [ ]:

# transform = transforms.Compose([transforms.ToTensor(),# this convert in 0-1 for each channel
#                               transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
# both parameters are “Sequences for each channel”. Color images have three channels (red, green, blue),
# therefore you need three parameters to normalize each channel.
# The first tuple (0.5, 0.5, 0.5) is the mean for all three channels 
# and the second (0.5, 0.5, 0.5) is the standard deviation for all three channels.

full_dataset = torch.utils.data.TensorDataset(Xt, yt)

# Splitting into Train and Val
train_size = int(0.8 * len(full_dataset))
val_size   = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# train_dataset.transforms = transform
# val_dataset.transforms = transform

# Creating  data_loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False)

it, lt = next(iter(train_loader))
print(it.shape,lt.shape)

iv, lv = next(iter(val_loader))
print(iv.shape,lv.shape)

In [19]:
# UNET model

class double_conv(nn.Module):
    """(conv => BN => ReLU) * 2"""

    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(nn.MaxPool2d(2), double_conv(in_ch, out_ch))

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2))
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.up1 = up(384, 256, False)
        self.up2 = up(128, 64, False)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x = self.up1(x3, x2)
        x = self.up2(x, x1)
        x = self.outc(x)
        return torch.sigmoid(x)

In [26]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
  
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)

    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model


In [27]:
model = get_unet((3,128,128))

NameError: name 'Conv2D' is not defined

In [21]:
from torchsummary import summary
summary(model,(3,256,256))

RuntimeError: Given transposed=1, weight of size [192, 192, 2, 2], expected input[2, 256, 64, 64] to have 192 channels, but got 256 channels instead

In [ ]:
lr = 0.01
optimizer =  torch.optim.SGD(model.parameters(), lr=lr,momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [94]:
# lr = 0.01
# optimizer =  torch.optim.SGD(model.parameters(), lr=lr,momentum=0.9)
# criterion = nn.BCEWithLogitsLoss()
# criterion = BCEDiceLoss(eps=1.0, activation=None)

In [95]:
from tqdm.auto import tqdm as tq

In [ ]:
n_epochs = 10

val_loss_min = np.Inf 

train_loss_list = []
val_loss_list =   []
for epoch in range(1, n_epochs+1):
    print("Enter the dragon")
    train_loss = 0.0
    val_loss = 0.0
    model.train()
    for data, target in train_loader:
        print(data.size())
        print(target.size())
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data).to(torch.long)
        #output = output.long()
        print(output.dtype, target.dtype)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
          
    # validate the model #
    model.eval()
    for data, target in val_loader:
        
        #output = output.long()
        data = data.to(device)
        target = target.to(device)
        output = model(data).to(torch.long)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_X)
    val_loss = val_loss/len(val_X)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, val_loss))
    
    # save model if validation loss has decreased
    if val_loss <= val_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        val_loss_min,
        val_loss))

#     torch.save(model.state_dict(), 'model_cifar.pt')
    val_loss_min = val_loss

    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

# *****END CODE